In [1]:
# Packages
import pandas as pd
import numpy as np
import sklearn
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn import metrics
import seaborn as sns
import matplotlib.pyplot as plt
from pprint import pprint

In [2]:
# Import game level stats for GK and FP
## 2017 and 2020 dropped from project. Inconsistent statistics available for 2017, No all star game in 2020
S22_GK_stats = pd.read_csv('C:/Users/scott/OneDrive/BANA/Capstone/Data/S22_GK_stats.csv')
S21_GK_stats = pd.read_csv('C:/Users/scott/OneDrive/BANA/Capstone/Data/S21_GK_stats.csv')
#S20_GK_stats = pd.read_csv('C:/Users/scott/OneDrive/BANA/Capstone/Data/S20_GK_stats.csv')
S19_GK_stats = pd.read_csv('C:/Users/scott/OneDrive/BANA/Capstone/Data/S19_GK_stats.csv')
S18_GK_stats = pd.read_csv('C:/Users/scott/OneDrive/BANA/Capstone/Data/S18_GK_stats.csv')
# S17_GK_stats = pd.read_csv('C:/Users/scott/OneDrive/BANA/Capstone/Data/S17_GK_stats.csv')
S22_FP_stats = pd.read_csv('C:/Users/scott/OneDrive/BANA/Capstone/Data/S22_FP_stats.csv')
S21_FP_stats = pd.read_csv('C:/Users/scott/OneDrive/BANA/Capstone/Data/S21_FP_stats.csv')
#S20_FP_stats = pd.read_csv('C:/Users/scott/OneDrive/BANA/Capstone/Data/S20_FP_stats.csv')
S19_FP_stats = pd.read_csv('C:/Users/scott/OneDrive/BANA/Capstone/Data/S19_FP_stats.csv')
S18_FP_stats = pd.read_csv('C:/Users/scott/OneDrive/BANA/Capstone/Data/S18_FP_stats.csv')
# S17_FP_stats = pd.read_csv('C:/Users/scott/OneDrive/BANA/Capstone/Data/S17_FP_stats.csv')

In [3]:
GK_stats = pd.concat([S22_GK_stats,S21_GK_stats,S19_GK_stats,S18_GK_stats], ignore_index= True)
# GK_stats.to_csv('C:/Users/scott/OneDrive/BANA/Capstone/Data/GK_stats.csv') # save file
FP_stats = pd.concat([S22_FP_stats,S21_FP_stats,S19_FP_stats,S18_FP_stats], ignore_index= True)
# FP_stats.to_csv('C:/Users/scott/OneDrive/BANA/Capstone/Data/FP_stats.csv') # save file

In [4]:
# Isolate Y/N from start status
FP_stats['Start'] = FP_stats['Start'].str[0]
GK_stats['Start'] = GK_stats['Start'].str[0]
# Convert Start to bool
FP_stats['Start'] = FP_stats['Start'].replace({'Y':1,'N':0})
GK_stats['Start'] = GK_stats['Start'].replace({'Y':1,'N':0})
# Summarize team performance using match result scoring
FP_stats['Win_Loss_Draw'] = FP_stats['Win_Loss_Draw'].replace({'L':0, 'D':1, 'W':3})
GK_stats['Win_Loss_Draw'] = GK_stats['Win_Loss_Draw'].replace({'L':0, 'D':1, 'W':3})
# Drop stats that occur after All star game 
FP_stats = FP_stats[FP_stats['Before_All_Star'] == True]
GK_stats = GK_stats[GK_stats['Before_All_Star'] == True]

In [5]:
# Drop fields that are not needed
FP_stats = FP_stats.drop(['Unnamed: 0','Date','Day','Comp','Round','Venue','Result','Squad','Opponent','Pos','Match Report'], axis = 1)
GK_stats = GK_stats.drop(['Unnamed: 0','Date','Day','Comp','Round','Venue','Result','Squad','Opponent','Pos','Match Report'], axis = 1)

In [6]:
# Summarize player stats by ID, season, before all star game
## Aggregation dicts (could this be simplified by using tuples?)
    # Should more of the aggregations be mean instead of sum to account for...
FP_aggs = {'Start':'sum', 'Min':'sum', 'Gls':'sum', 'Ast':'sum', 'PK':'sum', 'PKatt':'sum', 'Sh':'sum', 'SoT':'sum', 'CrdY':'sum', 'CrdR':'sum', 'Touches':'sum', 'Press':'sum', 'Tkl':'sum', 'Int':'sum', 'Blocks':'sum', 'xG':'mean', 'npxG':'mean', 'xA':'mean', 'SCA':'sum', 'GCA':'sum', 'Cmp':'sum', 'Att':'sum', 'Cmp%':'mean', 'Prog':'sum', 'Carries':'sum', 'Prog.1':'sum', 'Succ':'sum', 'Att.1':'sum', 'Win_Loss_Draw':'sum'}
GK_aggs = {'Start':'sum', 'Min':'sum', 'SoTA':'sum', 'GA':'sum', 'Saves':'sum', 'Save%':'mean', 'CS':'sum', 'PSxG':'mean', 'PKatt':'sum', 'PKA':'sum', 'PKsv':'sum', 'PKm':'sum', 'Cmp':'sum', 'Att':'sum', 'Cmp%':'mean', 'Att.1':'sum', 'Thr':'sum', 'Launch%':'mean', 'AvgLen':'mean', 'Att.2':'sum', 'Launch%.1':'mean', 'AvgLen.1':'mean', 'Opp':'sum', 'Stp':'sum', 'Stp%':'mean', '#OPA':'sum', 'AvgDist':'mean', 'Win_Loss_Draw':'sum'}

FP_stats = FP_stats.groupby(['fbref_ID', 'Season']).agg(FP_aggs).reset_index()
GK_stats = GK_stats.groupby(['fbref_ID', 'Season']).agg(GK_aggs).reset_index()

In [7]:
# Add All-Star Indicator
AS_roster = pd.read_csv('C:/Users/scott/OneDrive/BANA/Capstone/Data/all_star_rosters.csv', encoding = 'iso-8859-1')
FP_stats = FP_stats.merge(AS_roster[['Year','fbref_ID']], how = 'left', left_on = ['Season','fbref_ID'], right_on = ['Year','fbref_ID']).rename(columns={'Year':'all_star'})
FP_stats['all_star'] = FP_stats['all_star'].fillna(False).replace([2018,2019,2021,2022],True) 
GK_stats = GK_stats.merge(AS_roster[['Year','fbref_ID']], how = 'left', left_on = ['Season','fbref_ID'], right_on = ['Year','fbref_ID']).rename(columns={'Year':'all_star'})
GK_stats['all_star'] = GK_stats['all_star'].fillna(False).replace([2018,2019,2021,2022],True) 

# Exploratory Data Analysis

In [8]:
# Address missing data
print(FP_stats.isnull().sum())
print(GK_stats.isnull().sum())
FP_stats = FP_stats.fillna(0)
GK_stats = GK_stats.fillna(0)

FP_stats.info()
GK_stats.info()
FP_stats.describe()

fbref_ID         0
Season           0
Start            0
Min              0
Gls              0
Ast              0
PK               0
PKatt            0
Sh               0
SoT              0
CrdY             0
CrdR             0
Touches          0
Press            0
Tkl              0
Int              0
Blocks           0
xG               1
npxG             1
xA               1
SCA              0
GCA              0
Cmp              0
Att              0
Cmp%             5
Prog             0
Carries          0
Prog.1           0
Succ             0
Att.1            0
Win_Loss_Draw    0
all_star         0
dtype: int64
fbref_ID         0
Season           0
Start            0
Min              0
SoTA             0
GA               0
Saves            0
Save%            0
CS               0
PSxG             0
PKatt            0
PKA              0
PKsv             0
PKm              0
Cmp              0
Att              0
Cmp%             0
Att.1            0
Thr              0
Launch%          0

,Season,Start,Min,Gls,Ast,PK,PKatt,Sh,SoT,CrdY,...,GCA,Cmp,Att,Cmp%,Prog,Carries,Prog.1,Succ,Att.1,Win_Loss_Draw
count,2347.000000,2347.000000,2347.000000,2347.000000,2347.000000,2347.000000,2347.000000,2347.000000,2347.000000,2347.000000,...,2347.000000,2347.000000,2347.000000,2347.000000,2347.000000,2347.000000,2347.000000,2347.000000,2347.000000,2347.000000
mean,2020.161483,9.510013,853.499361,1.356199,0.961227,0.118875,0.152109,12.153387,4.052407,1.845334,...,2.207925,346.507030,428.241585,78.035771,32.956966,356.918193,38.732850,9.077972,15.623349,17.654026
std,1.576657,6.956034,595.911507,2.387469,1.523819,0.506494,0.607813,14.226591,5.603348,1.943013,...,2.964016,297.793222,356.303334,10.345797,32.228237,287.236756,37.930375,10.807680,18.475521,11.148442
min,2018.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2019.000000,3.000000,297.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,0.000000,97.000000,124.000000,73.350000,8.000000,107.500000,9.000000,2.000000,3.000000,8.000000
50%,2021.000000,9.000000,808.000000,0.000000,0.000000,0.000000,0.000000,7.000000,2.000000,1.000000,...,1.000000,270.000000,344.000000,79.225000,24.000000,301.000000,28.000000,5.000000,9.000000,17.000000
75%,2022.000000,16.000000,1360.500000,2.000000,1.000000,0.000000,0.000000,17.000000,5.000000,3.000000,...,3.000000,545.500000,666.000000,84.433239,49.000000,541.000000,56.500000,13.000000,22.000000,26.000000
max,2022.000000,25.000000,2181.000000,24.000000,10.000000,5.000000,6.000000,105.000000,44.000000,11.000000,...,23.000000,1531.000000,1865.000000,100.000000,218.000000,1454.000000,292.000000,89.000000,154.000000,51.000000


In [9]:
FP_stats.groupby('all_star').mean()

,Season,Start,Min,Gls,Ast,PK,PKatt,Sh,SoT,CrdY,...,GCA,Cmp,Att,Cmp%,Prog,Carries,Prog.1,Succ,Att.1,Win_Loss_Draw
all_star,,,,,,,,,,,,,,,,,,,,,
False,2020.170591,9.171924,823.411373,1.192359,0.880942,0.091515,0.116393,11.269658,3.700133,1.795646,...,2.022212,333.352732,411.282097,78.041584,31.242559,341.838294,36.611728,8.501555,14.633496,17.087517
True,2019.947917,17.437500,1559.000000,5.197917,2.843750,0.760417,0.989583,32.875000,12.312500,3.010417,...,6.562500,654.947917,825.906250,77.899475,73.156250,710.510417,88.468750,22.593750,38.833333,30.937500


# Normalize data by season

In [10]:
S22_GK_stats = GK_stats[GK_stats['Season'] == 2022]
S21_GK_stats = GK_stats[GK_stats['Season'] == 2021]
S19_GK_stats = GK_stats[GK_stats['Season'] == 2019]
S18_GK_stats = GK_stats[GK_stats['Season'] == 2018]
S22_FP_stats = FP_stats[FP_stats['Season'] == 2022]
S21_FP_stats = FP_stats[FP_stats['Season'] == 2021]
S19_FP_stats = FP_stats[FP_stats['Season'] == 2019]
S18_FP_stats = FP_stats[FP_stats['Season'] == 2018]

In [11]:
def normalize_FP(df):
    result = df.copy()
    for feature_name in df.columns[2:31]:
        max_value = df[feature_name].max()
        min_value = df[feature_name].min()
        result[feature_name] = (df[feature_name] - min_value) / (max_value - min_value)
    return result

def normalize_GK(df):
    result = df.copy()
    for feature_name in df.columns[2:30]:
        max_value = df[feature_name].max()
        min_value = df[feature_name].min()
        result[feature_name] = (df[feature_name] - min_value) / (max_value - min_value)
    return result
# Is there a way to normalize by season without needing to split DF by season and then reassemble?

In [12]:
# Normalize all features by each season and FP/GK
S22_GK_stats = normalize_GK(S22_GK_stats)
S21_GK_stats = normalize_GK(S21_GK_stats)
S19_GK_stats = normalize_GK(S19_GK_stats)
S18_GK_stats = normalize_GK(S18_GK_stats)
S22_FP_stats = normalize_FP(S22_FP_stats)
S21_FP_stats = normalize_FP(S21_FP_stats)
S19_FP_stats = normalize_FP(S19_FP_stats)
S18_FP_stats = normalize_FP(S18_FP_stats)

# Join DFs back together
GK_stats = pd.concat([S22_GK_stats,S21_GK_stats,S19_GK_stats,S18_GK_stats], ignore_index= True)
FP_stats = pd.concat([S22_FP_stats,S21_FP_stats,S19_FP_stats,S18_FP_stats], ignore_index= True)

In [13]:
# Add Player details
demo = pd.read_csv('C:/Users/scott/OneDrive/BANA/Capstone/Data/player_demographics.csv', encoding = 'iso-8859-1')
FP_stats = FP_stats.merge(demo, how = 'left', on = ['Season','fbref_ID'])
GK_stats = GK_stats.merge(demo, how = 'left', on = ['Season','fbref_ID'])

In [14]:
FP_stats.groupby(['Season','Pos'])['Pos'].count()

Season  Pos 
2018    DF      173
        DFFW     12
        DFMF     20
        FW       79
        FWDF      6
        FWMF     71
        MF       87
        MFDF     15
        MFFW     55
2019    DF      188
        DFFW      6
        DFMF     10
        FW       74
        FWDF      8
        FWMF     84
        MF      103
        MFDF     14
        MFFW     52
2021    DF      217
        DFFW     10
        DFMF     17
        FW       87
        FWDF     11
        FWMF     95
        MF      110
        MFDF     14
        MFFW     65
2022    DF      222
        DFFW     15
        DFMF     20
        FW       94
        FWDF      6
        FWMF    101
        MF      149
        MFDF     12
        MFFW     45
Name: Pos, dtype: int64

In [15]:
# Add Model Indicators Based on role
## FP_stats.groupby(['Pos'])['Pos'].count()
FP_stats['DF_model'] = FP_stats['Pos'].str.contains('DF') == True
FP_stats['MF_model'] = FP_stats['Pos'].str.contains('MF') == True
FP_stats['FW_model'] = FP_stats['Pos'].str.contains('FW') == True

In [ ]:
# Rename Features to useful names

In [16]:
FP_stats.groupby(['FW_model','MF_model','DF_model'])['fbref_ID'].count()
# No players fit in all models, but a large number cross into two
# Especially forwards and midfielders. This is expected based on role. Expect to see similar features of importance for both models

FW_model  MF_model  DF_model
False     False     True        800
          True      False       449
                    True        122
True      False     False       334
                    True         74
          True      False       568
Name: fbref_ID, dtype: int64

In [27]:
#FP_stats['fbref_ID']/
FP_stats.groupby(['all_star','fbref_ID']).agg({'fbref':'count'})


KeyError: "Column(s) ['fbref'] do not exist"

In [47]:
FP_FW = FP_stats[FP_stats['FW_model'] == True]
FP_FW.columns
X = FP_FW.iloc[:,1:31]
Y = FP_FW['all_star']

best_features = SelectKBest(score_func=chi2, k=3)
fit = best_features.fit(X,Y)
feature_scores = pd.concat([pd.DataFrame(X.columns),pd.DataFrame(fit.scores_)], axis=1)
feature_scores.columns = ['Feature','Score']
feature_scores.sort_values(by = 'Score', ascending= False)

,Feature,Score
6,PKatt,55.785465
5,PK,55.520549
3,Gls,44.957585
8,SoT,33.890660
20,GCA,28.861829
7,Sh,26.373452
19,SCA,26.215596
4,Ast,25.092099
1,Start,20.735409
2,Min,19.682147


In [48]:
FP_MF = FP_stats[FP_stats['MF_model'] == True]
X = FP_MF.iloc[:,1:31]
Y = FP_MF['all_star']

best_features = SelectKBest(score_func=chi2, k=3)
fit = best_features.fit(X,Y)
feature_scores = pd.concat([pd.DataFrame(X.columns),pd.DataFrame(fit.scores_)], axis=1)
feature_scores.columns = ['Feature','Score']
feature_scores.sort_values(by = 'Score', ascending= False)

,Feature,Score
5,PK,41.189506
6,PKatt,38.080389
20,GCA,23.827087
4,Ast,23.618313
3,Gls,22.018051
19,SCA,21.101128
24,Prog,18.143882
26,Prog.1,16.844832
27,Succ,16.392735
8,SoT,16.122562


In [50]:
FP_DF = FP_stats[FP_stats['DF_model'] == True]
X = FP_DF.iloc[:,1:31]
Y = FP_DF['all_star']

best_features = SelectKBest(score_func=chi2, k=3)
fit = best_features.fit(X,Y)
feature_scores = pd.concat([pd.DataFrame(X.columns),pd.DataFrame(fit.scores_)], axis=1)
feature_scores.columns = ['Feature','Score']
feature_scores.sort_values(by = 'Score', ascending= False)

,Feature,Score
11,Touches,8.638991
14,Int,8.619210
2,Min,8.549674
1,Start,8.382009
29,Win_Loss_Draw,8.200730
22,Att,8.159182
21,Cmp,7.865983
25,Carries,7.265018
24,Prog,6.089866
13,Tkl,5.445409


In [51]:
X = GK_stats.iloc[:,1:30]
Y = GK_stats['all_star']

best_features = SelectKBest(score_func=chi2, k=3)
fit = best_features.fit(X,Y)
feature_scores = pd.concat([pd.DataFrame(X.columns),pd.DataFrame(fit.scores_)], axis=1)
feature_scores.columns = ['Feature','Score']
feature_scores.sort_values(by = 'Score', ascending= False)

,Feature,Score
7,CS,7.004316
28,Win_Loss_Draw,4.482085
13,Cmp,3.251909
14,Att,2.763481
2,Min,2.751364
1,Start,2.712633
20,Att.2,2.682043
16,Att.1,2.579158
24,Stp,2.523619
23,Opp,1.990390


# Modeling

### Algorithms to consider
* Random Forest
* Other Ensemble (decision tree, AdaBoost)
* Neural Network
* Logistic (Need variable selection by model)
* SVM
* KNN?

### Notes
* Test/Train Split should needs to be adjusted 
    * What should be my test criteria? Should season be excluded for testing prediction on new season?
    * How can I assure that test partition has sufficient number of allstar records

In [35]:
# Split into Test/Train
fw_train, fw_test, fw_as_train, fw_as_test = train_test_split(FP_FW.iloc[:,2:31], FP_FW['all_star'], test_size=0.2, random_state=1017, stratify=FP_FW['all_star'])
mf_train, mf_test, mf_as_train, mf_as_test = train_test_split(FP_MF.iloc[:,2:31], FP_MF['all_star'], test_size=0.2, random_state=623, stratify=FP_MF['all_star'])
df_train, df_test, df_as_train, df_as_test = train_test_split(FP_DF.iloc[:,2:31], FP_DF['all_star'], test_size=0.2, random_state=225, stratify=FP_DF['all_star'])
gk_train, gk_test, gk_as_train, gk_as_test = train_test_split(GK_stats.iloc[:,2:30], GK_stats['all_star'], test_size=0.2, random_state=511, stratify=GK_stats['all_star'])

In [ ]:
print(fw_train.shape, mf_train.shape, df_train.shape, gk_train.shape)

### Random Forest

In [36]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=1000, max_features='sqrt', bootstrap=False)
pprint(clf.get_params())

{'bootstrap': False,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'sqrt',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 1000,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}


In [43]:
clf.fit(fw_train, fw_as_train)
y_pred = clf.predict(fw_test)

print('Random Forest FW Accuracy:',metrics.f1_score(fw_as_test, y_pred))

clf.fit(mf_train, mf_as_train)
y_pred = clf.predict(mf_test)

print('Random Forest MF Accuracy:',metrics.f1_score(mf_as_test, y_pred))

clf.fit(df_train, df_as_train)
y_pred = clf.predict(df_test)

print('Random Forest DF Accuracy:',metrics.f1_score(df_as_test, y_pred))

clf.fit(gk_train, gk_as_train)
y_pred = clf.predict(gk_test)

print('Random Forest GK Accuracy:',metrics.f1_score(gk_as_test, y_pred))

Random Forest FW Accuracy: 0.4347826086956522
Random Forest MF Accuracy: 0.0


C:\Users\scott\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Random Forest DF Accuracy: 0.14285714285714285
Random Forest GK Accuracy: 0.6666666666666666


#### Random Forest Notes
* Consider running decision tree and evaluate if interpretability is worth accuracy trade off
* CV using cross_val_score and RepeatedStratifiedKFold

### AdaBoost

In [38]:
from sklearn.ensemble import AdaBoostClassifier
ab = AdaBoostClassifier(n_estimators=1000, learning_rate=1)
pprint(ab.get_params())

{'algorithm': 'SAMME.R',
 'base_estimator': None,
 'learning_rate': 1,
 'n_estimators': 1000,
 'random_state': None}


In [42]:
ab_model = ab.fit(fw_train, fw_as_train)
y_pred = ab.predict(fw_test)
print('AdaBoost FW Accuracy:',metrics.f1_score(fw_as_test, y_pred))

ab_model = ab.fit(mf_train, mf_as_train)
y_pred = ab.predict(mf_test)
print('AdaBoost MF Accuracy:',metrics.f1_score(mf_as_test, y_pred))

ab_model = ab.fit(df_train, df_as_train)
y_pred = ab.predict(df_test)
print('AdaBoost DF Accuracy:',metrics.f1_score(df_as_test, y_pred))

ab_model = ab.fit(gk_train, gk_as_train)
y_pred = ab.predict(gk_test)
print('AdaBoost GK Accuracy:',metrics.f1_score(gk_as_test, y_pred))

AdaBoost FW Accuracy: 0.10526315789473685
AdaBoost MF Accuracy: 0.25
AdaBoost DF Accuracy: 0.18181818181818182
AdaBoost GK Accuracy: 0.0


### XGBoost

In [ ]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error



### Neural Network

In [40]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(solver='lbfgs')

pprint(clf.get_params())

{'activation': 'relu',
 'alpha': 0.0001,
 'batch_size': 'auto',
 'beta_1': 0.9,
 'beta_2': 0.999,
 'early_stopping': False,
 'epsilon': 1e-08,
 'hidden_layer_sizes': (100,),
 'learning_rate': 'constant',
 'learning_rate_init': 0.001,
 'max_fun': 15000,
 'max_iter': 200,
 'momentum': 0.9,
 'n_iter_no_change': 10,
 'nesterovs_momentum': True,
 'power_t': 0.5,
 'random_state': None,
 'shuffle': True,
 'solver': 'lbfgs',
 'tol': 0.0001,
 'validation_fraction': 0.1,
 'verbose': False,
 'warm_start': False}


In [44]:
clf.fit(fw_train, fw_as_train)
y_pred = clf.predict(fw_test)
print('NN FW Accuracy:',metrics.f1_score(fw_as_test, y_pred))

clf.fit(mf_train, mf_as_train)
y_pred = clf.predict(mf_test)
print('NN MF Accuracy:',metrics.f1_score(mf_as_test, y_pred))

clf.fit(df_train, df_as_train)
y_pred = clf.predict(df_test)
print('NN DF Accuracy:',metrics.f1_score(df_as_test, y_pred))
# Defender model does not consistently converge

clf.fit(gk_train, gk_as_train)
y_pred = clf.predict(gk_test)
print('NN GK Accuracy:',metrics.f1_score(gk_as_test, y_pred))

NN FW Accuracy: 0.5517241379310345
NN MF Accuracy: 0.14285714285714285


C:\Users\scott\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


NN DF Accuracy: 0.19047619047619047
NN GK Accuracy: 0.3333333333333333


# Misc

In [ ]:
import sklearn.metrics
from math import sqrt
train_preds = knn_model.predict(fw_test)
mse = mean_squared_error(fw_as_train, train_preds)
rmse = sqrt(mse)
rmse

In [ ]:
# Find Optimal K
from sklearn.model_selection import GridSearchCV
parameters = {"n_neighbors": range(1, 50)}
gridsearch = GridSearchCV(KNeighborsRegressor(), parameters)
gridsearch.fit(fw_train, fw_as_train)
print('Ideal K', gridsearch.best_params_)
print('KNN FW Accuracy:',metrics.accuracy_score(fw_as_test, train_preds))

In [ ]:
# DFs for individual seasons
S22_FP = FP_stats[FP_stats['Season'] == 2022]
S22_GK = GK_stats[GK_stats['Season'] == 2022]
S21_FP = FP_stats[FP_stats['Season'] == 2021]
S21_GK = GK_stats[GK_stats['Season'] == 2021]
S19_FP = FP_stats[FP_stats['Season'] == 2019]
S19_GK = GK_stats[GK_stats['Season'] == 2019]
S18_FP = FP_stats[FP_stats['Season'] == 2018]
S18_GK = GK_stats[GK_stats['Season'] == 2018]

In [ ]:
# Save Stats to file
FP_stats.to_csv('C:/Users/scott/OneDrive/BANA/Capstone/Data/FP_stats.csv')
GK_stats.to_csv('C:/Users/scott/OneDrive/BANA/Capstone/Data/GK_stats.csv')

In [ ]:
# Linear Regression
regressor = LinearRegression()
regressor.fit(GK_stats.iloc[:,2:30],GK_stats['all_star'])

In [ ]:
print(regressor.coef_)
print(regressor.intercept_)

### Additional Features Needed?

* Update Field Names to be interpretable 
* consider enhancing start feature to be % start or new feature for games played

### Grouping

* most stats should be summed, review to confirm

### Parameters for all star game
* breakdown by position (check year by year)
* primarily used for final conclusions

### Models
* Start with full model that considers all field players. Will always need a separate model for GK due to different set of metrics
* Build sub-models for position groups (fw, mid, def, gk)?
    * Would need separate groupings to handle players that split across positions and roles like 'wing'